In [1]:
%%writefile skin_cancer_app.py
import os
import cv2
import re
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing import image
import csv
from datetime import datetime

def find_last_conv_layer(model):
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            return layer.name
    raise ValueError("convolutional layer not found")

def compute_gradcam(model, img_tensor, last_conv_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0,1))
    conv_outputs = conv_outputs[0].numpy()
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:,:,i] *= pooled_grads[i]
    heatmap = conv_outputs.mean(axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) else 1
    return heatmap

def overlay_heatmap(heatmap, orig_img, alpha=0.4):
    heatmap = cv2.resize(np.uint8(255*heatmap), (orig_img.shape[1], orig_img.shape[0]))
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return cv2.addWeighted(orig_img, 1-alpha, heatmap, alpha, 0)


def create_model():
    base_model = MobileNet()
    x = base_model.layers[-6].output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.25)(x)
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = create_model()
model.load_weights('best_model.h5')

class_names = ['Actinic Keratoses', 'Basal cell carcinoma', 'Benign keratosis', 'Dermatofibroma', 'Melanoma', 'Melanocytic nevi', 'Vascular skin lesion']

def predict_skin_cancer(img):
    img = image.img_to_array(img)
    img = tf.image.resize(img, (224, 224))
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    preds = model.predict(img)[0]
    confidences = {class_names[i]: float(preds[i]) for i in range(7)}
    return confidences


xray_model = tf.keras.models.load_model("best_xray_detailed.h5")
xray_classes = ['Normal', 'Virus', 'Bacteria']

def predict_xray_with_gradcam(pil_img):
    arr_orig = cv2.resize(np.array(pil_img), (224,224)).astype('uint8')
    x = tf.image.resize(arr_orig, (224,224))
    x = preprocess_input(x)
    inp = np.expand_dims(x, 0)

    preds = xray_model.predict(inp)[0]
    confs = {xray_classes[i]: float(preds[i]) for i in range(len(xray_classes))}

    last_conv = find_last_conv_layer(xray_model)
    heatmap   = compute_gradcam(xray_model, inp, last_conv)
    overlay   = overlay_heatmap(heatmap, arr_orig)

    return confs, overlay


xray_brain_model = tf.keras.models.load_model('best_model_brain_tumor.h5')
xray_brain_classes = ['Glioma Tumor', 'Meningioma Tumor', 'No Tumor', 'Pituitary Tumor']
def predict_brain_with_gradcam(pil_img):
    arr_orig = cv2.resize(np.array(pil_img), (224,224)).astype('uint8')
    x = arr_orig.astype('float32') / 255.0
    inp = np.expand_dims(x, 0)

    preds = xray_brain_model.predict(inp)[0]
    confs = {xray_brain_classes[i]: float(preds[i]) for i in range(len(xray_brain_classes))}

    last_conv = find_last_conv_layer(xray_brain_model)
    heatmap   = compute_gradcam(xray_brain_model, inp, last_conv)
    overlay   = overlay_heatmap(heatmap, arr_orig)

    return confs, overlay



def is_valid_email(email):
    email_regex = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"
    return re.match(email_regex, email) is not None
def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False
def book_appointment(name, email, doctor, date, time_str, additional_info):
    if not (name and email and doctor and date and time_str):
        return "❗ Խնդրում ենք լրացնել բոլոր դաշտերը։"
    if not is_valid_email(email):
        return "❗ Անվավեր էլ-փոստի հասցե։"
    if not is_valid_date(date):
        return "❗ Դիտարկվող օրը պետք է լինի YYYY-MM-DD ձևաչափով։"
    doctor_schedules = {
        "Ալինա Ալինանյան": ("10:00", "16:00"),
        "Գուրգեն Հակոբյան": ("12:00", "18:00"),
        "Բժ. Սամվեն Մարտինյան": ("09:00", "15:00")
    }
    schedule = doctor_schedules.get(doctor, None)
    if schedule is None:
        return "❗ Ընտրված բժիշկի մասին տեղեկատվություն չկա։"
    start, end = schedule
    try:
        booking_time = datetime.strptime(time_str.strip(), "%H:%M").time()
        start_time = datetime.strptime(start, "%H:%M").time()
        end_time = datetime.strptime(end, "%H:%M").time()
        if not (start_time <= booking_time <= end_time):
            return f"❗ {doctor}-ը աշխատում է միայն {start}-ից {end} միջակայքում։"
    except Exception as e:
        return "❗ Ժամը պետք է լինի HH:MM ձևաչափով։"
    conflict_found = False
    if os.path.exists("appointments.csv"):
        with open("appointments.csv", mode="r", newline='') as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) >= 6:
                    _, _, _, existing_doctor, existing_date, existing_time, *_ = row
                    if (existing_doctor == doctor and existing_date == date and existing_time == time_str):
                        conflict_found = True
                        break
    if conflict_found:
        return f"❗ Բժիշկ {doctor}-ի մոտ, {date} ժամը {time_str} արդեն ամրագրություն կա։"

    with open("appointments.csv", mode="a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow([datetime.now().isoformat(), name, email, doctor, date, time_str, additional_info])
    return f"✅ Այցելությունը նշանակված է {date} ժամը {time_str}-ում։ Շուտով կկապնվենք {email}-ին"

def send_message(email, message):
    if is_valid_email(email) and message:
        with open("messages.csv", mode="a", newline='') as f:
            writer = csv.writer(f)
            writer.writerow([datetime.now().isoformat(), email, message])
        return f"✅ Հաղորդագրությունն ուղարկվեց։ Մենք շուտով կպատասխանենք {email}-ին"
    return "❗ Խնդրում ենք ճիշտ լրացնել էլփոստի և նամակի դաշտերը։"

with gr.Blocks(title="Հիվանդությունների ավտոմատ ախտորոշում",
               css="""
                    .tab-content {padding: 20px;}
                    .gr-markdown h1, .gr-markdown p {text-align: center;}
                    .center-text {text-align: center;}
                    .gr-image {margin: auto;}
               """) as demo:
    gr.Markdown("<h1 class='center-text'>🩺 Հիվանդությունների հայտնաբերման վեբ հավելված</h1>")
    gr.Markdown("<p class='center-text'>Ներբեռնեք ձեր լուսանկարը և ուսումնասիրեք մեր կլինիկայի կողմից առաջարկվող ծառայությունները:</p>")

    with gr.Tabs():
        with gr.Tab("🔍 Կանխատեսել մաշկի հիվանդությունը"):
            gr.Markdown("<h2 class='center-text'>📷 Ներբեռնեք մաշկի նկարը</h2>")
            gr.Markdown("Խնդրում ենք ներբեռնել վնասված մաշկի հստակ պատկերը: Ընդունված ձևաչափեր՝ JPG, PNG:")
            with gr.Row():
                with gr.Column(scale=1, min_width=400):
                    img_input = gr.Image(label="Upload Image", type="pil", show_label=False, interactive=True)
                    predict_button = gr.Button("🔍 Կանխատեսել", variant="primary")
                    clear_button = gr.Button("❌ Մաքրել", variant="secondary")
                with gr.Column(scale=1, min_width=400):
                    prediction_output = gr.Label(num_top_classes=3, label="Արդյունքներ")
            predict_button.click(fn=predict_skin_cancer, inputs=img_input, outputs=prediction_output)

            def reset():
                return None, ""
            clear_button.click(fn=reset, inputs=[], outputs=[img_input, prediction_output])
            gr.Markdown("<hr>")
            gr.Markdown("<h3 class='center-text'>🧪 Փորձեք պատկերների օրինակներ</h3>")
            gr.Examples(
                examples=[
                    # ["/content/Melanoma.jpg"],
                    ["/content/Dermatofibroma1.jpg"],
                    ["/content/Melanocytic nevi.jpg"]
                ],
                inputs=[img_input],
                label="Պատկերների օրինակներ"
            )
            gr.Markdown("<h3 class='center-text'>Ի՞նչ է MobileNet-ը</h3>")
            gr.Markdown("""
            **MobileNet**-ը կոնվոլյուցիոն նեյրոնային ցանցերի (CNN) ընտանիք է, որը նախատեսված է պատկերների կլասիֆիկացիայի, օբյեկտների հայտնաբերման և այլ տեսողական ճանաչման խնդիրների համար։
            Դրանք նախագծված են փոքր չափերի,ցածր էներգիայի սպառման համար, ինչը դրան դարձնում է հարմար հաշվարկներ անելու համար՝  ռեսուրսներով սահմանափակ սարքերի, ինչպիսիք են սմարթֆոնները ։
            - Օգտագործում է **depthwise separable convolutions**
            - Հաճախ կիրառվում է բժշկական պատկերների կլասիֆիկացիայի համար:
            """)
            gr.Markdown("""
            **Ծանուցում** ⚠️
            Այս գործիքը միայն աջակցող է և չի փոխարինում բժշկի ախտորոշմանը։ Խնդրում ենք խորհրդակցել մասնագետ բժշկի հետ։
            """)
        with gr.Tab("🧠 Ուղեղի ուռուցքի ճանաչում"):
            gr.Markdown("<h2 class='center-text'>🧠 Վերբեռնեք ուղեղի ՄՌՏ պատկերը</h2>")
            gr.Markdown("Ներբեռնեք ՄՌՏ պատկեր՝ ուղեղի ուռուցքի հայտնաբերման համար:")
            with gr.Row():
                with gr.Column(scale=1, min_width=400):
                    brain_input = gr.Image(label="Բեռնել ՄՌՏ պատկեր", type="pil", show_label=False, interactive=True)
                    brain_button = gr.Button("🔍 Կանխատեսել", variant="primary")
                    brain_clear_button = gr.Button("❌ Մաքրել", variant="secondary")
                with gr.Column(scale=1, min_width=400):
                    # brain_output = gr.Label(num_top_classes=4, label="Արդյունքներ")
                    brain_output   = gr.Label(num_top_classes=4, label="Արդյունքներ")
                    brain_gradcam  = gr.Image(type="numpy", label="Grad-CAM Բացատրություն")
            # brain_button.click(fn=predict_brain_tumor, inputs=brain_input, outputs=brain_output)
            brain_button.click(
                fn=predict_brain_with_gradcam,
                inputs=brain_input,
                outputs=[brain_output, brain_gradcam]
            )
            # brain_clear_button.click(fn=lambda: (None, ""), inputs=[], outputs=[brain_input, brain_output])
            brain_clear_button.click(fn=lambda: (None, None, None), inputs=[], outputs=[brain_input, brain_output, brain_gradcam])
            gr.Markdown("<h3 class='center-text'>🧪 Փորձեք պատկերների օրինակներ</h3>")
            gr.Examples(
                examples=[

                    ["/content/glioma.jpg"],
                    ["/content/pituitary.jpg"]
                ],
                inputs=[brain_input],
                label="Օրինակներ"
            )
            gr.Markdown("""
            **Ծանուցում** ⚠️
            Այս գործիքը միայն աջակցող է և չի փոխարինում բժշկի ախտորոշմանը։ Խնդրում ենք խորհրդակցել մասնագետ բժշկի հետ։
            """)
        with gr.Tab("📚 Հիվանդությունների մասին"):
                gr.Markdown("### Մաշկային Հիվանդությունների Տեղեկատվություն", elem_id="disease_info_title")

                gr.Markdown("""
#### Բազալ բջջային կարցինոմա (Basal cell carcinoma)
Բազալ բջջային կարցինոմա, մաշկի քաղցկեղի ամենատարածված տիպն է։ Այն հաճախ ի հայտ է գալիս որպես մաշկից բարձր, անցավ մակերես՝ վրան մանր արյունատար անոթներ կամ այն կարող է բարձրացած, խոցոտումով շրջանի տեսքով լինել։ Դանդաղ է աճում, կարող է վնասել իր շուջը գտնվող հյուսվածքը, բայց հազվադեպ է տարածվում։

#### Սեբորեային կերատոզ (Benign keratosis)
Սեբորեային կերատոզը մաշկի ոչ քաղցկեղային ուռուցք է, որն առաջանում է էպիդերմիսի կերատինոցիտներից։ Մաշկի վրա հայտնվում են տարբեր գույներով՝ բաց շագանակագույնից մինչև սև։

#### Մելանոմա (Melanoma)
Չարորակ մելանոման մելանոցիտներից առաջացած ուռուցք է։ Սովորաբար առաջանում է մաշկում, բայց երբեմն նաև բերանում, աղիներում կամ աչքում։ Կարող է զարգանալ խալերից՝ չափերի մեծացման, գույնի փոփոխության կամ քորի դեպքում։

#### Ակտինիկ կերատոզ (Actinic keratosis)
Ակտինիկ կերատոզը մաշկի նախաքաղցկեղային խստացած, թեփոտ մակերես է։ Այն առաջանում է ուլտրամանուշակագույն ճառագայթման ազդեցության հետևանքով՝ առաջացնելով էպիդերմալ կերատինոցիտների փոփոխություններ։

#### Արյան անոթների անոմալիա (Vascular anomaly)
Արյան անոթների անոմալիաները կարող են տատանվել՝ սկսած ծննդյան նշանից մինչև մեծ ուռուցքներ։ Դրանք կարող են ի հայտ գալ ծննդյան պահին կամ տարիներ անց, երբեմն վնասվածքներից հետո։

#### Դերմատոֆիբրոմա (Dermatofibroma)
Դերմատոֆիբրոման մաշկի բարորակ հանգույց է, որը հաճախ ի հայտ է գալիս ոտքերին, արմունկներին կամ կրծքավանդակին։ Սովորաբար անհանգստություն չի պատճառում։

#### Մելանոցիտային նեւուս (Melanocytic nevus)
Խալը մաշկի գունավոր բջիջների՝ մելանոցիտների բարորակ աճ է։ Կարող է լինել մաշկի ներսում կամ վերևից տեսանելի գունավոր ձևավորում։
""")

        with gr.Tab("👨‍⚕️ Կլինիկայի աշխատակիցները"):
            gr.Markdown("<h2 class='center-text'>Ծանոթացեք մեր բժշկական թիմին</h2>")
            with gr.Row():

                with gr.Column():
                    gr.Image("/content/alice.jpg", label="Ալինա Ալինանյան")
                    gr.Markdown("<p class='center-text'><strong>Բժ. Ալինա Ալինանյան</strong>  \n Մաշկաբան, կոսմետոլոգ:</p>")
                    gr.Markdown("""
                    **Կրթություն**
                    - Երևանի Մխիթար Հերացու անվան պետական բժշկական համալսարան՝ Կոսմետոլոգի որակավորում
                    - Երևանի Մխիթար Հերացու անվան պետական ​​բժշկական համալսարան` Մաշկավեներաբանության օրդինատուրա
                    - Երևանի Մխիթար Հերացու անվան պետական ​​բժշկական համալսարան` Oրդինատորի աստիճան, Ալերգոլոգիա և իմունաբանություն
                    - Երևանի Մխիթար Հերացու անվան պետական ​​բժշկական համալսարան`Ընդհանուր բժշկություն
                    """)

                with gr.Column():
                    gr.Image("/content/bob.jpg", label="Գուրգեն Հակոբյան")
                    gr.Markdown("<p class='center-text'><strong>Բժ. Գուրգեն Հակոբյան</strong>  \n Մաշկաբան և AI մասնագետ:</p>")
                    gr.Markdown("""
                    **Կրթություն**
                    - Երևանի Մխիթար Հերացու անվան պետական բժշկական համալսարան, Ընդհանուր բժշկության ֆակուլտետ
                    - Երևանի պետական բժշկական համալսարան Մաշկաբանության և սեռավարակաբանության ամբիոնի օրդինատոր
                    """)
                with gr.Column():
                    gr.Image("/content/samvel.jpg", label="Բժ. Սամվեն Սամվելյան")
                    gr.Markdown("<p class='center-text'><strong>Բժ. Սամվեն Սամվելյան</strong>  \n Մաշկաբան:</p>")
                    gr.Markdown("""
                    **Կրթություն**
                    - Երևանի պետական բժշկական համալսարան, Մաշկաբանության և սեռավարակաբանության ամբիոն, կլինիկական օրդինատուրա
                    - Երևանի պետական բժշկական համալսարան, Մաշկաբանության և սեռավարակաբանության ամբիոն, ասպիրանտուրա
                    """)

        with gr.Tab("📨 Կապվեք մեր բժիշկների հետ"):
            gr.Markdown("<h3 class='center-text'>Ուղարկել նամակ</h3>")
            contact_email = gr.Textbox(label="Ձեր Email")
            contact_message = gr.Textbox(lines=4, label="Նամակ")
            send_btn = gr.Button("Ուղարկել")
            send_result = gr.Textbox(label="Արձագանք")
            send_btn.click(fn=send_message, inputs=[contact_email, contact_message], outputs=send_result)


        with gr.Tab("📅 Ամրագրեք այց բժիշկի մոտ"):
            gr.Markdown("<h3 class='center-text'>Ամրագրել այց</h3>")
            name_input = gr.Textbox(label="Անուն Ազգանուն")
            email_input = gr.Textbox(label="Email")
            doctor_dropdown = gr.Dropdown(
                choices=["Ալինա Ալինանյան", "Գուրգեն Հակոբյան", "Սամվեն Սամվելյան"],
                label="Ընտրեք բժիշկին"
            )
            date_input = gr.Textbox(label="Նախընտրելի ամսաթիվ (YYYY-MM-DD)")
            time_input = gr.Textbox(label="Նախընտրելի ժամ (օր. 14:30)")
            note_input = gr.Textbox(label="Լրացուցիչ տեղեկություններ (ընտրությամբ)", lines=3)
            book_button = gr.Button("Ամրագրել այց")
            book_result = gr.Textbox(label="Այցելության կարգավիճակ")
            book_button.click(fn=book_appointment,
                              inputs=[name_input, email_input, doctor_dropdown, date_input, time_input, note_input],
                              outputs=book_result)
        with gr.Tab("🩻 Թոքերի Ռենտգեն (X-ray)"):
            gr.Markdown("<h2 class='center-text'>🩻 Վերլուծեք X-ray նկարը</h2>")
            gr.Markdown("Բեռնեք թոքերի ռենտգեն պատկերը՝ որոշելու համար՝ արդյոք պատկերը նորմալ է, վիրուսային է թե բակտերիալ:")
            with gr.Row():
                with gr.Column(scale=1, min_width=400):
                    xray_input = gr.Image(label="Բեռնել X-ray նկարը", type="pil", show_label=False, interactive=True)
                    xray_button = gr.Button("🔍 Կանխատեսել", variant="primary")
                    xray_clear_button = gr.Button("❌ Մաքրել", variant="secondary")
                with gr.Column(scale=1, min_width=400):
                    # xray_output = gr.Label(num_top_classes=3, label="Արդյունքներ")
                    xray_output   = gr.Label(num_top_classes=3, label="Արդյունքներ")
                    xray_gradcam  = gr.Image(type="numpy", label="Grad-CAM Բացատրություն")
            # xray_button.click(fn=predict_xray, inputs=xray_input, outputs=xray_output)
            xray_button.click(
                fn=predict_xray_with_gradcam,
                inputs=xray_input,
                outputs=[xray_output, xray_gradcam]
            )
            # xray_clear_button.click(fn=lambda: (None, ""), inputs=[], outputs=[xray_input, xray_output])
            xray_clear_button.click(fn=lambda: (None, None, None), inputs=[], outputs=[xray_input, xray_output, xray_gradcam])
            gr.Markdown("<h3 class='center-text'>🧪 Փորձեք պատկերների օրինակներ</h3>")
            gr.Examples(
                examples=[
                    ["/content/pneumonia.png"],
                    ["/content/normal.png"],
                   # ["/content/xray_normal.jpg"]
                ],
                inputs=[xray_input],
                label="Օրինակային պատկերներ"
            )
            gr.Markdown("""
            **Ծանուցում** ⚠️
            Այս գործիքը միայն աջակցող է և չի փոխարինում բժշկի ախտորոշմանը։ Խնդրում ենք խորհրդակցել մասնագետ բժշկի հետ։
            """)


if __name__ == "__main__":
    demo.launch(share=True)


Writing skin_cancer_app.py


In [2]:
!pip install -q gradio tensorflow

In [3]:
!python skin_cancer_app.py

2025-08-01 22:41:00.542301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754088060.609439    1195 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754088060.634182    1195 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-01 22:41:00.711048: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-01 22:41:09.370601: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL